<a href="https://colab.research.google.com/github/tonmoycsedu/Cluster-Keywords/blob/master/Word_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''
Import Packages
'''
import gensim.models.keyedvectors as word2vec
from scipy.spatial.distance import cosine
import pandas as pd
import os
from os.path import isfile, join
import numpy as np
from numpy.linalg import norm
import gensim.downloader

In [3]:
'''
Initialize, load data
'''
language = "en"
df, model = None, None

model =  word2vec.KeyedVectors. \
                    load_word2vec_format('./word2vec_50k.bin', \
                                        binary=True, limit=50041)

gp_path = './'

In [4]:
'''
Load neutral words that define the groups
'''
male_words = []
with open(gp_path+"Gender - Male") as f:
    for line in f:
        male_words.append(line.split("\n")[0])

female_words = []
with open(gp_path+"Gender - Female") as f:
    for line in f:
        female_words.append(line.split("\n")[0])

In [5]:
'''
calculate bias direction when we have group of words
'''
def groupBiasDirection(gp1, gp2):
    print(gp1,gp2)
    dim = len(model["he"])
    g1,g2 = np.zeros((dim,), dtype=float), np.zeros((dim,), dtype=float)
    for p in gp1:
        p = p.strip()
        if p not in model:
            continue
        p_vec = model[p]/norm(model[p])
        g1 = np.add(g1,p_vec)

    for q in gp2:
        q = q.strip()
        if q not in model:
            continue
        q_vec = model[q]/norm(model[q])
        g2 = np.add(g2,q_vec) 

    g1, g2 = g1/norm(g1), g2/norm(g2)
    return (g1,g2)

In [6]:
g1, g2 = groupBiasDirection(male_words, female_words)

['he', 'son', 'his', 'him', 'father', 'man', 'boy', 'himself', 'male', 'brother', 'sons', 'fathers', 'men', 'boys', 'males', 'brothers', 'uncle', 'uncles', 'nephew', 'nephews'] ['she', 'daughter', 'hers', 'her', 'mother', 'woman', 'girl', 'herself', 'female', 'sister', 'daughters', 'mothers', 'women', 'girls', 'femen', 'sisters', 'aunt', 'aunts', 'niece', 'nieces']


In [7]:
'''
Compute Bias Score
'''

bias_scores = []
for w in list(model.vocab.keys()):
    bias_scores.append(round(cosine(g1,model[w])-cosine(g2,model[w]),4))

In [8]:
'''
Normalize Bias Score
'''

bias_min = min(bias_scores)
bias_max = max(bias_scores)

norm_bias_score = []

for b in bias_scores:
    if b<0:
        norm_bias_score.append(-1*b/bias_min)
    else:
        norm_bias_score.append(b/bias_max)

In [9]:
'''
Convert to a dataframe
'''

df = pd.DataFrame()
df['word'] = list(model.vocab.keys())
df['score'] = norm_bias_score
sorted = df.sort_values(by=['score'])

In [10]:
# Top 20 female words
sorted.iloc[-50:]

,word,score
14285,feminine,0.669542
50009,Elizabeth,0.670434
31591,womanhood,0.671921
38623,vixen,0.674598
16072,breastfeeding,0.678465
42354,doyenne,0.679060
8109,lesbian,0.680250
36786,shes,0.682927
28834,hysterectomy,0.682927
4785,nurse,0.685009
